In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import layers

import datetime
import tensorflow_addons as tfa 
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


## Load data

In [2]:
import read_rijksdata
MIN_NUM_ARTWORK = 50

In [3]:
img_folder = '/Users/erebor/Downloads/out_img'

In [4]:
images, labels_onehot, labels, names = read_rijksdata.load_data(MIN_NUM_ARTWORK=MIN_NUM_ARTWORK,
                                                 img_folder =img_folder,
                                                 labels_file='labels.txt',
                                                 names_file='names.txt')

Loading 112039 artworks...
 |███████████████████████████████████████-| 112038/112039 

Throwing away artwork with no artist...

Filtering for artists with at least 50 artworks...
 |████████████████████████████████████████| 6622/6622 

Dataset loaded!
images shape: (64697, 56, 56, 3)
labels shape: (64697,)
labels (one-hot): (64697, 366)
names shape: (64697, 1)


In [5]:
classes = len(list(set(labels)))
print('# of classes:',classes)

counts = pd.DataFrame(labels).value_counts()
print('Min # of artworks for all artists:',min(counts))
print('Min # of artworks specified:',MIN_NUM_ARTWORK)

# of classes: 366
Min # of artworks for all artists: 50
Min # of artworks specified: 50


## Download and compile model

In [6]:
input_shape = (56,56,3)
enet_kwargs = {'include_top':False,
               'weights':'imagenet',
               'input_tensor':None,
               'input_shape':input_shape,
               'pooling':None,
               'classes':classes,
               'classifier_activation':'softmax'}
enet_base = tf.keras.applications.efficientnet.EfficientNetB7(**enet_kwargs)
#enet_base.trainable = False

In [7]:
enet = tf.keras.models.Sequential()
enet.add(enet_base)
enet.add(tf.keras.layers.GlobalMaxPooling2D())
#enet.add(tf.keras.layers.Dropout(rate=0.01))
enet.add(tf.keras.layers.Dense(classes, activation="softmax"))

In [8]:
enet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 2, 2, 2560)        64097687  
_________________________________________________________________
global_max_pooling2d (Global (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 366)               937326    
Total params: 65,035,013
Trainable params: 64,724,286
Non-trainable params: 310,727
_________________________________________________________________


In [9]:
# loss function
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False,label_smoothing=0.0,name='categorical_crossentropy')

# metrics

TopKs = []
for k in [1,5]:
    TopK = tf.keras.metrics.TopKCategoricalAccuracy(k=k, name='top_{}'.format(k))
    TopKs.append(TopK)
metrics = ["acc"]
metrics.extend(TopKs)

f1 = tfa.metrics.F1Score(num_classes=classes, threshold=0.05)
metrics.append(f1)

# Optimizer
# very average Adam settings
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# compile it all
enet.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics)

## Train model

In [ ]:
history = enet.fit(x=images,y=labels_onehot,validation_split=.20, epochs=5)

Epoch 1/5
  18/1618 [..............................] - ETA: 44:19 - loss: 0.4353 - acc: 0.0000e+00 - top_1: 0.0000e+00 - top_5: 0.0040 - f1_score: 0.0000e+00

In [ ]:
# Save the weights
checkpoint_time = '{date:%Y-%m-%d_%H-%M}'.format(date=datetime.datetime.now())

save_file = './checkpoints/enet_{}'.format(checkpoint_time)
print('Saving to:',save_file)
enet.save_weights(save_file)

In [ ]:
enet.save('models/enet.h5')

## Re-load model and evaluation

In [ ]:
reconstructed_model = keras.models.load_model('models/enet.h5')

In [ ]:
allgood = np.testing.assert_allclose(enet.predict(X_), reconstructed_model.predict(X_))

In [ ]:
print(allgood)

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

confusion_matrix = sklearn.metrics.confusion_matrix(y_true=Y, y_pred=y_pred, labels=None)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(classes):
    fpr[i], tpr[i], _ = roc_curve(Y_train, y_pred)
    roc_auc[i] = auc(fpr[i], tpr[i])